# Soal Nomor 6
## Cara Menulis List ke File CSV

Berikut adalah cara untuk menulis list ["a","b","c"] ke file CSV bernama huruf.csv menggunakan Python.

### Menggunakan Modul CSV Built-in

Cara paling standar adalah menggunakan modul atau library `csv` yang sudah tersedia di Python:

In [15]:
import csv

# Data yang akan ditulis
data = ["a", "b", "c"]

# Menulis ke file CSV
with open('huruf.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Menulis sebagai satu baris
    writer.writerow(data)
    
print("File huruf.csv berhasil dibuat!")

# Membaca kembali untuk verifikasi
with open('huruf.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        print("Isi file:", row)

File huruf.csv berhasil dibuat!
Isi file: ['a', 'b', 'c']


# Soal Nomor 7
## Menangani Error saat Membaca File JSON dengan Format Tidak Valid

Ketika bekerja dengan file JSON, sering kali kita menghadapi masalah format yang tidak valid. Berikut adalah cara mengidentifikasi masalah dan solusinya:

### Jenis-jenis Error JSON yang Umum

1. **JSONDecodeError**: Format JSON tidak valid
2. **FileNotFoundError**: File tidak ditemukan
3. **PermissionError**: Tidak ada izin untuk membaca file
4. **UnicodeDecodeError**: Encoding file bermasalah

### Langkah 1: Membuat File JSON untuk Testing

Beberapa file JSON dengan format yang berbeda untuk testing:

In [16]:
import json
import os

# 1. Membuat file JSON yang valid
data_valid = {
    "nama": "John Doe",
    "umur": 30,
    "hobi": ["membaca", "menulis", "coding"]
}

with open('data_valid.json', 'w', encoding='utf-8') as file:
    json.dump(data_valid, file, indent=2, ensure_ascii=False)

# 2. Membuat file JSON yang tidak valid (missing quote)
invalid_json1 = '''
{
    "nama": "John Doe,
    "umur": 30,
    "hobi": ["membaca", "menulis"]
}
'''

with open('data_invalid1.json', 'w', encoding='utf-8') as file:
    file.write(invalid_json1)

# 3. Membuat file JSON yang tidak valid (extra comma)
invalid_json2 = '''
{
    "nama": "Jane Doe",
    "umur": 25,
    "hobi": ["menggambar", "musik",]
}
'''

with open('data_invalid2.json', 'w', encoding='utf-8') as file:
    file.write(invalid_json2)

# 4. Membuat file JSON yang tidak valid (unquoted keys)
invalid_json3 = '''
{
    nama: "Bob Smith",
    umur: 35,
    hobi: ["olahraga"]
}
'''

with open('data_invalid3.json', 'w', encoding='utf-8') as file:
    file.write(invalid_json3)

print("File-file JSON untuk testing telah dibuat:")
print("data_valid.json (format benar)")
print("data_invalid1.json (missing quote)")
print("data_invalid2.json (extra comma)")
print("data_invalid3.json (unquoted keys)")

File-file JSON untuk testing telah dibuat:
data_valid.json (format benar)
data_invalid1.json (missing quote)
data_invalid2.json (extra comma)
data_invalid3.json (unquoted keys)


### Langkah 2: Error Handling Dasar dengan Try-Except

Berikut adalah cara menangani error saat membaca file JSON:

In [17]:
def read_json_basic(filename):
    """Fungsi dasar untuk membaca JSON dengan error handling"""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
            print(f"Berhasil membaca {filename}")
            return data
    
    except FileNotFoundError:
        print(f"Error: File '{filename}' tidak ditemukan")
        return None
    
    except json.JSONDecodeError as e:
        print(f"Error JSON di {filename}: {e}")
        return None
    
    except PermissionError:
        print(f"Error: Tidak ada izin untuk membaca '{filename}'")
        return None
    
    except Exception as e:
        print(f"Error tidak terduga: {e}")
        return None

# Testing dengan berbagai file
print("=== Testing Error Handling Dasar ===")
files_to_test = [
    'data_valid.json',
    'data_invalid1.json', 
    'data_invalid2.json',
    'data_invalid3.json',
    'file_tidak_ada.json'
]

for filename in files_to_test:
    print(f"\nMembaca {filename}:")
    result = read_json_basic(filename)
    if result:
        print(f"Data: {result}")
    print("-" * 50)

=== Testing Error Handling Dasar ===

Membaca data_valid.json:
Berhasil membaca data_valid.json
Data: {'nama': 'John Doe', 'umur': 30, 'hobi': ['membaca', 'menulis', 'coding']}
--------------------------------------------------

Membaca data_invalid1.json:
Error JSON di data_invalid1.json: Invalid control character at: line 3 column 23 (char 25)
--------------------------------------------------

Membaca data_invalid2.json:
Error JSON di data_invalid2.json: Expecting value: line 5 column 36 (char 78)
--------------------------------------------------

Membaca data_invalid3.json:
Error JSON di data_invalid3.json: Expecting property name enclosed in double quotes: line 3 column 5 (char 7)
--------------------------------------------------

Membaca file_tidak_ada.json:
Error: File 'file_tidak_ada.json' tidak ditemukan
--------------------------------------------------
Error JSON di data_invalid1.json: Invalid control character at: line 3 column 23 (char 25)
-------------------------------

### Langkah 3: Solusi Perbaikan Otomatis

Kode perbaikan JSON yang rusak secara otomatis:

In [18]:
import re

def read_json_with_auto_fix(filename):
    """Membaca JSON dengan mencoba perbaikan otomatis untuk error umum"""
    
    def try_fix_json(content):
        """Mencoba memperbaiki JSON yang rusak"""
        fixes_applied = []
        
        # 1. Hapus trailing comma
        if re.search(r',\s*[}\]]', content):
            content = re.sub(r',(\s*[}\]])', r'\1', content)
            fixes_applied.append("Removed trailing commas")
        
        # 2. Tambahkan quotes untuk keys yang tidak quoted (sederhana)
        if re.search(r'{\s*\w+:', content):
            content = re.sub(r'(\{\s*)(\w+)(\s*:)', r'\1"\2"\3', content)
            content = re.sub(r'(,\s*)(\w+)(\s*:)', r'\1"\2"\3', content)
            fixes_applied.append("Added quotes to unquoted keys")
        
        # 3. Coba tambahkan quote yang hilang (sederhana)
        missing_quote_pattern = r':\s*([^",{\[\s]+[^",}\]\s]*)\s*[,}]'
        if re.search(missing_quote_pattern, content):
            content = re.sub(r':\s*([A-Za-z][A-Za-z\s]*[A-Za-z])\s*([,}])', r': "\1"\2', content)
            fixes_applied.append("Added missing quotes to string values")
        
        return content, fixes_applied
    
    try:
        # Coba baca terlebih normal dulu
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
            print(f"{filename} berhasil dibaca tanpa perbaikan")
            return data
            
    except json.JSONDecodeError as e:
        print(f"JSON error di {filename}, mencoba perbaikan otomatis...")
        print(f"Error asli: {e.msg} di baris {e.lineno}")
        
        try:
            # Baca sebagai text dan coba perbaiki
            with open(filename, 'r', encoding='utf-8') as file:
                content = file.read()
            
            print(f"Isi asli: {content[:100]}...")
            
            # Coba perbaikan
            fixed_content, fixes = try_fix_json(content)
            
            if fixes:
                print(f"Perbaikan yang diterapkan: {', '.join(fixes)}")
                print(f"Isi setelah perbaikan: {fixed_content[:100]}...")
                
                # Coba parse yang sudah diperbaiki
                data = json.loads(fixed_content)
                print(f"Berhasil setelah perbaikan!")
                
                # Simpan versi yang diperbaiki
                fixed_filename = filename.replace('.json', '_fixed.json')
                with open(fixed_filename, 'w', encoding='utf-8') as file:
                    json.dump(data, file, indent=2, ensure_ascii=False)
                print(f"File yang diperbaiki disimpan sebagai: {fixed_filename}")
                
                return data
            else:
                print("Tidak ada perbaikan yang bisa diterapkan")
                return None
                
        except json.JSONDecodeError as e2:
            print(f"Perbaikan gagal: {e2.msg}")
            return None
        except Exception as e2:
            print(f"Error saat perbaikan: {e2}")
            return None
    
    except Exception as e:
        print(f"Error lain: {e}")
        return None

# Testing perbaikan otomatis
print("=== Testing Perbaikan Otomatis ===")
files_to_fix = ['data_invalid1.json', 'data_invalid2.json', 'data_invalid3.json']

for filename in files_to_fix:
    print(f"\n{'='*60}")
    print(f"Mencoba perbaikan: {filename}")
    print('='*60)
    result = read_json_with_auto_fix(filename)
    if result:
        print(f"Data hasil perbaikan: {result}")
    print()

=== Testing Perbaikan Otomatis ===

Mencoba perbaikan: data_invalid1.json
JSON error di data_invalid1.json, mencoba perbaikan otomatis...
Error asli: Invalid control character at di baris 3
Isi asli: 
{
    "nama": "John Doe,
    "umur": 30,
    "hobi": ["membaca", "menulis"]
}
...
Perbaikan yang diterapkan: Added missing quotes to string values
Isi setelah perbaikan: 
{
    "nama": "John Doe,
    "umur": 30,
    "hobi": ["membaca", "menulis"]
}
...
Perbaikan gagal: Invalid control character at


Mencoba perbaikan: data_invalid2.json
JSON error di data_invalid2.json, mencoba perbaikan otomatis...
Error asli: Expecting value di baris 5
Isi asli: 
{
    "nama": "Jane Doe",
    "umur": 25,
    "hobi": ["menggambar", "musik",]
}
...
Perbaikan yang diterapkan: Removed trailing commas, Added missing quotes to string values
Isi setelah perbaikan: 
{
    "nama": "Jane Doe",
    "umur": 25,
    "hobi": ["menggambar", "musik"]
}
...
Berhasil setelah perbaikan!
File yang diperbaiki disimpan sebag

### Best Practices dan Kesimpulan

#### Best Practices untuk Error Handling JSON:

1. **Selalu gunakan try-except** saat membaca file JSON
2. **Berikan pesan error yang jelas** untuk memudahkan debugging
3. **Validasi data** setelah berhasil dibaca
4. **Gunakan encoding yang tepat** (biasanya utf-8)
5. **Log error** untuk analisis lebih lanjut
6. **Buat backup** sebelum melakukan perbaikan otomatis

#### Strategi Penanganan Error:

- **Preventif**: Validasi format sebelum menyimpan
- **Reaktif**: Handle error dengan gracefully
- **Recovery**: Coba perbaikan otomatis untuk error umum
- **Fallback**: Berikan nilai default atau alternatif